In [0]:
!pip install deeppavlov

In [0]:
!python -m deeppavlov install ner_conll2003

In [0]:
config = {
    "dataset_reader": {
        "class_name": "conll2003_reader",
        "data_path": "./onthology_objects_data_new/dataset_1/",
        "dataset_name": "conll2003",
        "provide_pos": False
    },
    "dataset_iterator": {
        "class_name": "data_learning_iterator",
        "seed": 42
    },
    "chainer": {
    "in": ["x"],
    "in_y": ["y"],
    "pipe": [
      {
        "in": ["x"],
        "class_name": "lazy_tokenizer",
        "out": ["x_tokens"]
      },
      {
        "in": ["x_tokens"],
        "class_name": "str_lower",
        "out": ["x_lower"]
      },
      {
        "in": ["x_lower"],
        "class_name": "sanitizer",
        "nums": True,
        "out": ["x_san"]
      },
      {
        "in": ["x_san"],
        "id": "word_vocab",
        "class_name": "simple_vocab",
        "pad_with_zeros": True,
        "special_tokens": ["<UNK>"],
        "fit_on": ["x_san"],
        "save_path": "{MODELS_PATH}/onthology_objects/word.dict",
        "load_path": "{MODELS_PATH}/onthology_objects/word.dict",
        "out": ["x_tok_ind"]
      },
      {
        "in": ["y"],
        "id": "tag_vocab",
        "class_name": "simple_vocab",
        "pad_with_zeros": True,
        "fit_on": ["y"],
        "save_path": "{MODELS_PATH}/onthology_objects/tag.dict",
        "load_path": "{MODELS_PATH}/onthology_objects/tag.dict",
        "out": ["y_ind"]
      },
      {
        "in": ["x_tokens"],
        "class_name": "char_splitter",
        "out": ["x_char"]
      },
      {
        "in": ["x_char"],
        "id": "char_vocab",
        "class_name": "simple_vocab",
        "pad_with_zeros": True,
        "fit_on": ["x_char"],
        "save_path": "{MODELS_PATH}/onthology_objects/char.dict",
        "load_path": "{MODELS_PATH}/onthology_objects/char.dict",
        "out": ["x_char_ind"]
      },
      {
        "in": ["x_tokens"],
        "class_name": "mask",
        "out": ["mask"]
      },
      {
        "in": ["x_san"],
        "id": "glove_emb",
        "class_name": "glove",
        "pad_zero": True,
        "load_path": "{DOWNLOADS_PATH}/embeddings/glove.6B.100d.txt",
        "out": ["x_emb"]
      },
      {
        "id": "embeddings",
        "class_name": "emb_mat_assembler",
        "embedder": "#glove_emb",
        "vocab": "#word_vocab"
      },
      {
        "id": "embeddings_char",
        "class_name": "emb_mat_assembler",
        "character_level": True,
        "emb_dim": 32,
        "embedder": "#glove_emb",
        "vocab": "#char_vocab"
      },
      {
        "id": "capitalization",
        "class_name": "capitalization_featurizer",
        "in": ["x_tokens"],
        "out": ["cap"]
      },
      {
        "in": ["x_emb", "mask", "x_char_ind", "cap"],
        "in_y": ["y_ind"],
        "out": ["y_predicted"],
        "class_name": "ner",
        "main": True,
        "token_emb_dim": "#glove_emb.dim",
        "n_hidden_list": [128],
        "net_type": "rnn",
        "cell_type": "lstm",
        "use_cudnn_rnn": True,
        "n_tags": "#tag_vocab.len",
        "capitalization_dim": "#capitalization.dim",
        "char_emb_dim": "#embeddings_char.dim",
        "save_path": "{MODELS_PATH}/onthology_objects/model_no_pos",
        "load_path": "{MODELS_PATH}/onthology_objects/model_no_pos",
        "char_emb_mat": "#embeddings_char.emb_mat",
        "two_dense_on_top": True,
        "use_crf": True,
        "use_batch_norm": True,
        "embeddings_dropout": True,
        "top_dropout": True,
        "intra_layer_dropout": True,
        "l2_reg": 0,
        "learning_rate": 1e-2,
        "dropout_keep_prob": 0.5
      },
      {
        "ref": "tag_vocab",
        "in": ["y_predicted"],
        "out": ["tags"]
      }
    ],
    "out": ["x_tokens", "tags"]
  },
    "train": {
        "epochs": 100,
        "batch_size": 64,
        "metrics": [
            {
                "name": "ner_f1",
                "inputs": [
                    "y",
                    "tags"
                ]
            }
        ],
        "validation_patience": 7,
        "val_every_n_epochs": 1,
        "log_every_n_epochs": -1,
        "show_examples": False,
        "validate_best": True,
        "test_best": True
    },
    "metadata": {
        "variables": {
            "ROOT_PATH": ".",
            "DOWNLOADS_PATH": "{ROOT_PATH}/downloads",
            "MODELS_PATH": "{ROOT_PATH}/models"
        },
        "requirements": [
            "{DEEPPAVLOV_PATH}/requirements/gensim.txt",
            "{DEEPPAVLOV_PATH}/requirements/tf.txt"
        ],
        "labels": {
            "telegram_utils": "NERCoNLL2003Model",
            "server_utils": "NER"
        },
        "download": [
            {
                "url": "http://files.deeppavlov.ai/embeddings/glove.6B.100d.txt",
                "subdir": "{DOWNLOADS_PATH}/embeddings"
            }
        ]
    }
}

In [0]:
import json

with open("config.json", "w") as fout:
  json.dump(config, fout, ensure_ascii=False, indent=2)

In [24]:
!python -m deeppavlov download config.json

2018-12-10 08:18:56.491 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 208: Starting new HTTP connection (1): files.deeppavlov.ai
2018-12-10 08:18:56.886 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 396: http://files.deeppavlov.ai:80 "GET /embeddings/glove.6B.100d.txt.md5 HTTP/1.1" 200 52
2018-12-10 08:18:56.890 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 208: Starting new HTTP connection (1): files.deeppavlov.ai
2018-12-10 08:18:57.254 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 396: http://files.deeppavlov.ai:80 "GET /embeddings/glove.6B.100d.txt HTTP/1.1" 200 None
2018-12-10 08:18:57.255 INFO in 'deeppavlov.core.data.utils'['utils'] at line 63: Downloading from http://files.deeppavlov.ai/embeddings/glove.6B.100d.txt to /content/downloads/embeddings/glove.6B.100d.txt
347MB [00:30, 11.4MB/s]


In [0]:
from deeppavlov import build_model, train_model

In [53]:
!wget http://files.deeppavlov.ai/datasets/onthology_objects_data_new.zip

--2018-12-10 08:39:10--  http://files.deeppavlov.ai/datasets/onthology_objects_data_new.zip
Resolving files.deeppavlov.ai (files.deeppavlov.ai)... 93.175.29.74
Connecting to files.deeppavlov.ai (files.deeppavlov.ai)|93.175.29.74|:80... connected.
HTTP request sent, awaiting response... 200 OK
Cookie coming from files.deeppavlov.ai attempted to set domain to lnsigo.mipt.ru
Length: 346661 (339K) [application/zip]
Saving to: ‘onthology_objects_data_new.zip’

onthology_objects_d 100%[===================>] 338.54K   469KB/s    in 0.7s    

2018-12-10 08:39:11 (469 KB/s) - ‘onthology_objects_data_new.zip’ saved [346661/346661]



In [54]:
! unzip onthology_objects_data_new.zip

Archive:  onthology_objects_data_new.zip
   creating: onthology_objects_data_new/
   creating: onthology_objects_data_new/dataset_1/
  inflating: onthology_objects_data_new/dataset_1/test.txt  
  inflating: onthology_objects_data_new/dataset_1/train.txt  
  inflating: onthology_objects_data_new/dataset_1/valid.txt  
   creating: onthology_objects_data_new/dataset_2/
  inflating: onthology_objects_data_new/dataset_2/test.txt  
  inflating: onthology_objects_data_new/dataset_2/train.txt  
  inflating: onthology_objects_data_new/dataset_2/valid.txt  
   creating: onthology_objects_data_new/dataset_3/
  inflating: onthology_objects_data_new/dataset_3/test.txt  
  inflating: onthology_objects_data_new/dataset_3/train.txt  
  inflating: onthology_objects_data_new/dataset_3/valid.txt  


In [55]:
model = train_model(config)

2018-12-10 08:39:18.459 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 100: [loading vocabulary from /content/models/onthology_objects/word.dict]
2018-12-10 08:39:18.625 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 89: [saving vocabulary to /content/models/onthology_objects/word.dict]
2018-12-10 08:39:18.637 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 100: [loading vocabulary from /content/models/onthology_objects/tag.dict]
2018-12-10 08:39:19.105 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 89: [saving vocabulary to /content/models/onthology_objects/tag.dict]
2018-12-10 08:39:19.108 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 100: [loading vocabulary from /content/models/onthology_objects/char.dict]
2018-12-10 08:39:19.631 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 89: [saving vocabulary to /content/models/onthology_objects/char.dict]
2018-12-10 0

{"valid": {"eval_examples_count": 218, "metrics": {"ner_f1": 8.5014}, "time_spent": "0:00:02", "epochs_done": 0, "batches_seen": 0, "train_examples_seen": 0, "impatience": 0, "patience_limit": 7}}


2018-12-10 08:40:48.676 INFO in 'deeppavlov.core.commands.train'['train'] at line 571: New best ner_f1 of 18.2116
2018-12-10 08:40:48.678 INFO in 'deeppavlov.core.commands.train'['train'] at line 573: Saving model
2018-12-10 08:40:48.679 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 52: [saving model to /content/models/onthology_objects/model_no_pos]


{"valid": {"eval_examples_count": 218, "metrics": {"ner_f1": 18.2116}, "time_spent": "0:00:52", "epochs_done": 1, "batches_seen": 28, "train_examples_seen": 1771, "impatience": 0, "patience_limit": 7}}


2018-12-10 08:41:37.387 INFO in 'deeppavlov.core.commands.train'['train'] at line 571: New best ner_f1 of 52.1502
2018-12-10 08:41:37.388 INFO in 'deeppavlov.core.commands.train'['train'] at line 573: Saving model
2018-12-10 08:41:37.390 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 52: [saving model to /content/models/onthology_objects/model_no_pos]


...

2018-12-10 09:18:43.129 INFO in 'deeppavlov.core.commands.train'['train'] at line 578: Did not improve on the ner_f1 of 75.0863
2018-12-10 09:18:43.130 INFO in 'deeppavlov.core.commands.train'['train'] at line 589: Ran out of patience


{"valid": {"eval_examples_count": 218, "metrics": {"ner_f1": 74.7535}, "time_spent": "0:38:47", "epochs_done": 43, "batches_seen": 1204, "train_examples_seen": 76153, "impatience": 7, "patience_limit": 7}}


2018-12-10 09:18:43.441 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 100: [loading vocabulary from /content/models/onthology_objects/word.dict]
2018-12-10 09:18:43.457 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 100: [loading vocabulary from /content/models/onthology_objects/tag.dict]
2018-12-10 09:18:43.459 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 100: [loading vocabulary from /content/models/onthology_objects/char.dict]
2018-12-10 09:18:43.462 INFO in 'deeppavlov.models.embedders.glove_embedder'['glove_embedder'] at line 52: [loading GloVe embeddings from `/content/downloads/embeddings/glove.6B.100d.txt`]
2018-12-10 09:18:43.463 INFO in 'gensim.models.keyedvectors'['keyedvectors'] at line 204: loading projection weights from /content/downloads/embeddings/glove.6B.100d.txt
2018-12-10 09:18:43.465 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'rb', 'uri': '/content/downloa

{"valid": {"eval_examples_count": 218, "metrics": {"ner_f1": 75.0863}, "time_spent": "0:00:02"}}
{"test": {"eval_examples_count": 223, "metrics": {"ner_f1": 73.8269}, "time_spent": "0:00:02"}}


2018-12-10 09:19:24.829 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 100: [loading vocabulary from /content/models/onthology_objects/word.dict]
2018-12-10 09:19:24.850 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 100: [loading vocabulary from /content/models/onthology_objects/tag.dict]
2018-12-10 09:19:24.853 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 100: [loading vocabulary from /content/models/onthology_objects/char.dict]
2018-12-10 09:19:24.857 INFO in 'deeppavlov.models.embedders.glove_embedder'['glove_embedder'] at line 52: [loading GloVe embeddings from `/content/downloads/embeddings/glove.6B.100d.txt`]
2018-12-10 09:19:24.859 INFO in 'gensim.models.keyedvectors'['keyedvectors'] at line 204: loading projection weights from /content/downloads/embeddings/glove.6B.100d.txt
2018-12-10 09:19:24.861 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'rb', 'uri': '/content/downloa

In [56]:
!python -m deeppavlov evaluate config.json

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!
2018-12-10 09:21:55.111 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 100: [loading vocabulary from /content/models/onthology_objects/word.dict]
2018-12-10 09:21:55.124 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 100: [loading vocabulary from /content/models/onthology_objects/tag.dict]
2018-12-10 09:21:55.126 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 100: [loading vocabulary from /content/models/onthology_objects